# Billboard Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
import urllib
from collections import Counter

################################################################################
## General Stuff
################################################################################
from searchUtils import findNearest
from searchUtils import findExt
from searchUtils import findPatternExt
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from time import sleep

################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb



from multiArtist import multiartist
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-10-04 22:03:42.934161


# Global Vars

# Starter Class Section

In [2]:
basedir = "/Volumes/Piggy/Charts/"
try:
    filename = glob(join(basedir, "data", "billboard", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

#### Extra

In [ ]:
chartData  = {}
dirname = None
baseURL = "https://www.billboard.com"
for iul,ul in enumerate(fdata.findAll("ul")):
    lis = ul.findAll("li", {"class": "chart-group__item"})
    for j,li in enumerate(lis):  
        a = li.find('a')
        if a is not None:
            href = a['href']
            text = a.text
            subdir = getDirname(href)[1:]
            chartData[text] = [baseURL, subdir, getBasename(href)]

In [ ]:
#chartData

# Download Yearly Information

In [ ]:
from time import sleep
years = [str(x) for x in range(1958,2014)]
years = ['2019']
for year in years:
    url="https://www.billboard.com/archive/charts/{0}".format(year)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    savedir = join(basedir, "data", "billboard", "yearly")
    savename = join(savedir, "{0}.p".format(year))
    if isFile(savename):
        continue
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(1)

# Parse Yearly Information

In [ ]:
downloads = {}
baseURL = "https://www.billboard.com"
for ifile in sorted(glob(join(basedir, "data", "billboard", "yearly", "*.p"))):
    year = getBaseFilename(ifile)
    if downloads.get(year) is None:
        downloads[year] = {}
    fdata = getHTML(ifile)
    for iul,ul in enumerate(fdata.findAll("ul")):
        lis = ul.findAll("li", {"class": "chart-group__item"})
        for j,li in enumerate(lis):  
            a = li.find('a')
            if a is not None:
                href = a['href']
                text = a.text
                subdir = getDirname(href)[1:]
                downloads[year][href] = [baseURL, subdir, getBasename(href)]

In [ ]:
downloads.keys()

# Download Category Data

In [ ]:
baseURL = "https://www.billboard.com"

for year, yearData in downloads.items():   
    for href, hrefData in yearData.items():
        url  = "{0}/{1}".format(baseURL, href)
        year = getBasename(hrefData[1])
        category = hrefData[2]

        savedir  = join(basedir, "data", "billboard", "categories")
        savename = join(savedir, "{0}-{1}.p".format(year, category))

        if isFile(savename):
            continue

        print("  Trying to download and save {0}".format(savename))
        try:

            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,} 

            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need
        except:
            continue

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)        

# Parse Category Data

In [ ]:
data = {}

baseURL = "https://www.billboard.com"
names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
categories = set([x[5:] for x in names])

albumCategories  = [x for x in categories if x.endswith("albums")]
songCategories   = [x for x in categories if not x.endswith("albums")]


names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
years = sorted(set([x[:4] for x in names]))

print("There are {0} years".format(len(years)))
print("There are {0} charts".format(len(categories)))
print("There are {0} album charts".format(len(albumCategories)))
print("There are {0} song charts".format(len(songCategories)))

In [ ]:
#categories

In [125]:
catdir = join(basedir, "data", "billboard", "categories")
files = findPatternExt(catdir, pattern='2019', ext='.p')    
for ifile in files:
    bsdata = getHTML(ifile)
    name   = bsdata.find("h2", {"class": "simple-page__title"})
    print(ifile,'\t',name)

/Volumes/Piggy/Charts/data/billboard/categories/2019-ALT.p 	 <h2 class="simple-page__title">Alternative Albums - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ARG.p 	 <h2 class="simple-page__title"> - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ARK.p 	 <h2 class="simple-page__title">Hot Rock Songs - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ASI.p 	 <h2 class="simple-page__title">Adult Contemporary - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ASX.p 	 <h2 class="simple-page__title">Holiday Airplay - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ATF.p 	 <h2 class="simple-page__title">Adult Top 40 - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ATS.p 	 <h2 class="simple-page__title">Billboard Artist 100 - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-BGR.p 	 <h2 class="simple-page__title">Bluegrass Albums - 2

/Volumes/Piggy/Charts/data/billboard/categories/2019-RKT.p 	 <h2 class="simple-page__title">Rock Digital Song Sales - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-RLP.p 	 <h2 class="simple-page__title">Top Rap Albums - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-ROS.p 	 <h2 class="simple-page__title">Rock Streaming Songs - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-RTT.p 	 <h2 class="simple-page__title">Mainstream Rock - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-SLL.p 	 <h2 class="simple-page__title">Top Gospel Albums - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-SOC.p 	 <h2 class="simple-page__title">Social 50 - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-STM.p 	 <h2 class="simple-page__title">Streaming Songs - 2019 Archive</h2>
/Volumes/Piggy/Charts/data/billboard/categories/2019-STX.p 	 <h2 class="simple-page__title">Soun

In [122]:
bsdata.findAll("h2")

[<h2>Follow Billboard</h2>,
 <h2>About Billboard</h2>,
 <h2 class="simple-page__title">Alternative Albums - 2019 Archive</h2>,
 <h2>Magazine</h2>,
 <h2>Sharing Billboard</h2>,
 <h2>Explore Billboard</h2>,
 <h2>Information</h2>]

In [123]:
bsdata.find("h2", {"class": "simple-page__title"})

#Alternative Albums - 2019 Archive</h2>
#            ")

<h2 class="simple-page__title">Alternative Albums - 2019 Archive</h2>

In [ ]:
def parseBillboardFile(ifile):
    fdata = getHTML(ifile)
    data  = {}
    for i,table in enumerate(fdata.findAll('table')):
        ths = table.findAll("th")
        trs = table.findAll("tr")

        headers = [x.text for x in ths]
        for j,tr in enumerate(trs[1:]):
            tds  = tr.findAll('td')
            vals = [x.text for x in tds]
            #print(headers,vals)

            if len(vals) == 3:
                date,name,artist = vals
                try:
                    date = getDateTime(", ".join([date, year]))
                except:
                    print(vals)
                    print(", ".join([date, year]))

                    1/0
            elif len(vals) == 1:
                date = vals[0]
                try:
                    date = getDateTime(", ".join([date, year]))
                except:
                    print(vals)
                    1/0

            if not isDate(date):
                raise ValueError("Could not form date for {0}".format(date))

            date = printDateTime(date)
            if data.get(date) is not None:
                raise ValueError("Already seen this date!!!")
            data[date] = {"Artist": artist, "Name": name}
            
            #print("{0: <12}{1}".format(date, data[date]))
            continue
            
            
            if data.get(date) is None:
                data[date] = {}
            #print("{0: <20}{1: <20}{2: <20}{3: <20}".format(artist, name, category, date))
            #continue
            if data.get(artist) is None:
                data[artist] = {}
                #print("Artist: {0}".format(artist))
            if data[artist].get(name) is None:
                data[artist][name] = {}
                #print("\tSong: {0} ({1})".format(name, len(data[artist])))
            if data[artist][name].get(category) is None:
                data[artist][name][category] = []
            data[artist][name][category].append([date])

    return data

In [ ]:
catdir = join(basedir, "data", "billboard", "categories")
for year in years:
    data  = {}
    files = findPatternExt(catdir, pattern=year, ext='.p')    
    for ifile in files:
        chart   = getBaseFilename(ifile)
        chart   = "-".join(chart.split('-')[1:])
        results = parseBillboardFile(ifile)
        data[chart] = results
        print("{0: <10}{1: <30}{2}".format(year, chart, len(results)))
        
    savedir = join(basedir, "data", "billboard", "results")
    savename = join(savedir, "{0}.p".format(year))
    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)

# Aggregrate Charts

In [39]:
class billboardCharts:
    def __init__(self):

        self.adult =  ['adult-contemporary','adult-pop-songs']
        self.adult += ["ASI", "ATF"]

        self.alternative =  ['alternative-albums', 'alternative-songs']
        self.alternative += ["MRT", "ALT"]

        self.folkblue = ['americana-folk-albums', 'bluegrass-albums', 'blues-albums']
        self.folkblue = ["BLU", "BGR", "FLK"]

        self.christian  = ['christian-airplay', 'hot-christian-songs', 'christian-albums', 'christian-digital-song-sales', 'christian-songs', 'christian-streaming-songs']
        self.christian += ['gospel-airplay', 'gospel-albums', 'gospel-digital-song-sales', 'gospel-songs', 'gospel-streaming-songs']
        self.christian += ["CRI", "CRT", "ICO", "ILL", "SLL", "GOS", "GSI", "GSS", "CHS", "GDT"]

        self.countryMusic  = ['country-airplay', 'country-albums', 'country-digital-song-sales', 'country-songs', 'country-streaming-songs']
        self.countryMusic += ["CSA", "CSI", "CST", "CLP", "CDT"]

        self.electronic  = ['dance-club-play-songs', 'dance-electronic-albums', 'dance-electronic-digital-song-sales', 'dance-electronic-songs', 'dance-electronic-streaming-songs']
        self.electronic += ["DAN", "DAS", "DDT", "BSI", "BST", "DSI", "ELP"]

        self.hot  = ['hot-100', 'pop-songs', 'radio-songs']
        self.hot += ['streaming-songs', 'rhythmic-40', 'heatseekers-albums']    
        self.top  = ['billboard-200', 'artist-100', 'top-album-sales']
        self.hot += self.top
        self.hot += ["TLN", "HSB", "HSI", "TLP", "TSL", "TFM", "ATS"]

        self.rnb  = ['r-and-b-albums', 'r-and-b-hip-hop-digital-song-sales', 'r-and-b-hip-hop-streaming-songs', 'r-and-b-songs', 'r-b-hip-hop-albums', 'r-b-hip-hop-songs']
        self.rnb += ['rap-albums','rap-song', 'hot-r-and-b-hip-hop-airplay']
        self.rnb += ["RLP", "RAP", "RBL", "RBM", "RBS", "RBT", "BLP"]

        self.rock  = ['rock-airplay','hot-mainstream-rock-tracks', 'rock-albums','rock-digital-song-sales','rock-songs','rock-streaming-songs', 'hard-rock-albums']
        self.rock += ["RCK", "RKA", "RKT", "ROS", "RTT", "MTL", "ARK"]

        self.canadian = ['canadian-albums','canadian-hot-100']
        self.canadian += ["CAN", "CNA"]
        self.latin    = ['latin-airplay','latin-albums','latin-digital-song-sales','latin-pop-albums','latin-pop-songs','latin-songs','latin-streaming-songs']
        self.latin   += ["HTA", "HTL", "LSS", "LDT", "LPO", "LPP", "LCM"]
        self.mexican  = ['regional-mexican-albums','regional-mexican-songs']
        self.mexican += ["LRM", "LMX"]
        self.world    = ['china-v-chart', 'japan-hot-100','tropical-albums', 'tropical-songs','world-albums', 'reggae-albums']
        self.world   += ["LSA", "LTS", "JPN", "WLP", "JAH", "ARG"]

        self.holiday  = ['holiday-albums','holiday-season-digital-song-sales','holiday-songs','holiday-streaming-songs']
        self.holiday += ['XSS', "XML", "XDT", "ASX"]

        self.classical  = ['classical-albums']
        self.classical += ['COA']

        self.jazz  = ['jazz-albums', 'jazz-songs']
        self.jazz += ['JLS', 'JSI']

        self.comedy  = ['comedy-albums']
        self.comedy += ['GIG']

        self.newage  = ['new-age-albums']
        self.newage += ['NLP']

        self.general = ['digital-albums','digital-song-sales']
        self.general = ['HDS', 'DLP']

        self.kids  = ['kids-albums']
        self.kids += ['KID']

        self.twitter  = ['twitter-emerging-artists', 'twitter-top-tracks', 'social-50']
        self.twitter += ['STM', 'SOC']

        self.vinyl  = ['vinyl-albums', "catalog-albums", 'tastemaker-albums']
        self.vinyl += ["TAS", "VNL", "TLC"]

        self.rare  = ['independent-albums']
        self.rare += ['IND']

        self.soundtrack  = ['soundtracks']
        self.soundtrack += ["STX"]
        
        self.chartNames = self.__dict__.keys()
        
    def getCharts(self, name=None):        
        charts = []
        if name is None:
            print("  Getting All Charts")
            for chart in self.chartNames:
                charts += self.__dict__[chart]
        else:
            print("  Getting Chart For {0}".format(name))
            if name in self.__dict__.keys():
                charts += self.__dict__[name]
        if len(charts) == 0:
            raise ValueError("Could not find any charts")
        print("  Using {0} Charts".format(len(charts)))
        return charts
    
    
from artistIgnores import getArtistIgnores
def getArtistRenames():
    artistRenames = getFile("masterRename.yaml")
    return artistRenames

In [47]:
class billboardData:
    def __init__(self, minYear=None, maxYear=None, country=None, debug=False):
        debug=False    
        
        self.basedir  = "/Volumes/Piggy/Charts/"
        self.basename = "Billboard"
        
        self.bc = billboardCharts()
        self.charts = []
        self.files = []
        self.findFiles()
            
        self.minYear   = minYear
        self.maxYear   = maxYear
        
        self.artistRenames   = {}

        self.chartData       = {}
        self.fullChartData   = {}
        self.artistAlbumData = {}
        

    def saveFullChartData(self):
        print("Saving {0} Full Artist Data".format(len(self.fullChartData)))
        saveFile(idata=self.fullChartData, ifile="current{0}FullChartArtistAlbumData.p".format(self.basename), debug=True)        
        
    def saveArtistAlbumData(self):
        print("Saving {0} Artist Album Data".format(len(self.artistAlbumData)))
        saveFile(idata=self.artistAlbumData, ifile="current{0}ArtistAlbumData.p".format(self.basename), debug=True)

        
        
    def setRenames(self, artistRenames):
        self.artistRenames = artistRenames   
        
    def getArtistAlbumData(self):
        return self.artistAlbumData
    
    def getFullChartData(self):
        return self.fullChartData
    
    def setChartUsage(self, name=None):
        charts = ['adult', 'alternative', 'folkblue', 'christian', 'countryMusic', 'electronic', 'hot', 'top', 'rnb', 'rock',
                  'canadian', 'latin', 'mexican', 'world', 'holiday', 'classical', 'jazz', 'comedy', 'newage', 'general',
                  'kids', 'twitter', 'vinyl', 'rare', 'soundtrack']
        
        
        if name is not None:
            self.charts += self.bc.getCharts(name)
        else:
            self.charts = self.bc.getCharts(None)
        if name is None:
            name = "None"
        print("  Using Charts ({0}): {1}".format(name, self.charts))
        
    
                
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        
    def findFiles(self):
        savedir = join(self.basedir, "data", "billboard", "results")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} files.".format(len(files)))

    
    def setFullChartData(self):        
        renameStats  = Counter()
        chartCounter = Counter()
        
        for ifile in files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
                
                for date, dResults in cnameResults.items():
                    if self.minYear is not None:
                        if getDateTime(date).year < int(self.minYear):
                            continue
                    if self.maxYear is not None:
                        if getDateTime(date).year > int(self.maxYear):
                            continue
                    stryear = getDateTime(date).year

                    artist = dResults["Artist"]

                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist   

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue


                    chartCounter[chartName] += 1

                    album  = dResults["Name"]

                    if self.chartData.get(artist) is None:
                        self.chartData[artist] = Counter()
                    self.chartData[artist][album] += 1
                    
                    if self.fullChartData.get(artist) is None:
                        self.fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if self.fullChartData[artist][key].get(album) is None:
                        self.fullChartData[artist][key][album] = {}
                    if self.fullChartData[artist][key][album].get(chartName) is None:
                        self.fullChartData[artist][key][album][chartName] = {}
                    self.fullChartData[artist][key][album][chartName][date] = 0
                #print("{0: <40}{1}".format("{0}-{1}".format(chartName,stryear),len(self.fullChartData)))

In [51]:
def getArtistRenames():
    artistRenames = getFile("masterRename.yaml")
    return artistRenames

from artistIgnores import getArtistIgnores
bd = billboardData(minYear=1, maxYear=20000)
bd.setRenames(getArtistRenames())
bd.setChartUsage("rock")
bd.setFullChartData()
bd.setArtistAlbumData()

bd.saveArtistAlbumData()
bd.saveFullChartData()

_, _ = clock("Last Run")

Found 62 files.
  Getting Chart For rock
  Using 14 Charts
  Using Charts (rock): ['rock-airplay', 'hot-mainstream-rock-tracks', 'rock-albums', 'rock-digital-song-sales', 'rock-songs', 'rock-streaming-songs', 'hard-rock-albums', 'RCK', 'RKA', 'RKT', 'ROS', 'RTT', 'MTL', 'ARK']
Saving 510 Artist Album Data
Saving data to currentBillboardArtistAlbumData.p
  --> This file is 23.7kB.
Saved data to currentBillboardArtistAlbumData.p
  --> This file is 23.7kB.
Saving 510 Full Artist Data
Saving data to currentBillboardFullChartArtistAlbumData.p
  --> This file is 72.1kB.
Saved data to currentBillboardFullChartArtistAlbumData.p
  --> This file is 72.1kB.
Current Time is Sun Oct 04, 2020 23:18:20 for Last Run


****

In [126]:


    def setFullChartData(self):
        fullChartData = {}
        renameStats   = Counter()


        charts = []

        if albums:
            charts = [x for x in charts if x.endswith("albums")]


        print("Charts: {0}".format(charts))
        return

        artistRenames = self.getArtistRenames()
        renameStats   = Counter()
        fullChartData = {}
        chartData     = {}
        #country       = "USA"
        country       = country
        chartCounter  = Counter()



                    if False:
                        if chartName in nonMusic:
                            continue        
                        if chartName in mixMusic:
                            continue
                        if chartName.startswith("gospel"):
                            continue
                        if chartName.startswith("bluegrass"):
                            continue
                        if chartName.startswith("classical"):
                            continue
                        if chartName.startswith("jazz"):
                            continue
                        if chartName.startswith("holiday"):
                            continue
                        if chartName.startswith("comedy"):
                            continue
                        if chartName.startswith("soundtracks"):
                            continue
                        if chartName.startswith("new-age"):
                            continue
                        if chartName.startswith("heatseekers"):
                            continue
                        if chartName.startswith("dance"):
                            continue
                        if chartName.startswith("christian"):
                            continue
                        if chartName.startswith("kids"):
                            continue
                        if country in ["USA", "US"]:
                            if chartName in nonUS:
                                continue

                    artist = dResults["Artist"]

                    renamedArtist = artist
                    for testArtist in artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist   

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue


                    chartCounter[chartName] += 1

                    album  = dResults["Name"]

                    if chartData.get(artist) is None:
                        chartData[artist] = Counter()
                    chartData[artist][album] += 1




                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = 0
            print(len(fullChartData))



        artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in fullChartData.items()}        

        if artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)

        saveFile(idata=chartCounter, ifile="chartCounter.p", debug=True)
        saveFile(idata=chartData, ifile="chartData.p", debug=True)


        saveFile(idata=artistAlbumData, ifile="currentBillboardArtistAlbumData.p", debug=True)
        saveFile(idata=fullChartData, ifile="currentBillboardFullChartArtistAlbumData.p", debug=True)

        _, _ = clock("Last Run")        


SyntaxError: invalid syntax (<ipython-input-126-d6a92449ce97>, line 171)

In [ ]:


artistRenames = getArtistRenames()
from fullCharts import fullCharts
fCharts = fullCharts(t40Charts, ctype=None, country=["US", "Canada"], minYear=1, maxYear=2015)
fCharts.setRenames(artistRenames)
fCharts.setFullChartData()
fCharts.setArtistAlbumData()
artistAlbumData = fCharts.getArtistAlbumData()
saveFile(idata=artistAlbumData, ifile="currentTop40ArtistAlbumData.p", debug=True)
fullChartData = fCharts.getFullChartData()
saveFile(idata=fullChartData, ifile="currentTop40FullChartArtistAlbumData.p", debug=True)

_, _ = clock("Last Run")

In [190]:
from collections import Counter
def findArtistData(minYear=2019, maxYear=3000, country=None, albums=False):

    ### Charts
    useAdult        = False
    useAlternative  = False
    useFolkBlue     = False
    useChristian    = False
    useCountryMusic = False
    useElectronic   = False
    useHot          = True
    useRnB          = False
    useRock         = False
    useNonUS        = False
    useHoliday      = False
    useClassical    = False
    useJazz         = False
    useComedy       = False
    useNewAge       = False
    useGeneral      = False
    useKids         = False
    useTwitter      = False
    useOther        = False
    useTest         = False
    useRare         = False
    useVinyl        = False
    useSountrack    = False

    charts = []

    adult =  ['adult-contemporary','adult-pop-songs']
    adult += ["ASI", "ATF"]
    if useAdult:
        charts += adult

    alternative =  ['alternative-albums', 'alternative-songs']
    alternative += ["MRT", "ALT"]
    if useAlternative:
        charts += alternative

    folkblue = ['americana-folk-albums', 'bluegrass-albums', 'blues-albums']
    folkblue = ["BLU", "BGR", "FLK"]
    if useFolkBlue:
        charts += folkblue

    christian  = ['christian-airplay', 'hot-christian-songs', 'christian-albums', 'christian-digital-song-sales', 'christian-songs', 'christian-streaming-songs']
    christian += ['gospel-airplay', 'gospel-albums', 'gospel-digital-song-sales', 'gospel-songs', 'gospel-streaming-songs']
    christian += ["CRI", "CRT", "ICO", "ILL", "SLL", "GOS", "GSI", "GSS", "CHS", "GDT"]

    if useChristian:
        charts += christian

    countryMusic  = ['country-airplay', 'country-albums', 'country-digital-song-sales', 'country-songs', 'country-streaming-songs']
    countryMusic += ["CSA", "CSI", "CST", "CLP", "CDT"]
    if useCountryMusic:
        charts += countryMusic

    electronic  = ['dance-club-play-songs', 'dance-electronic-albums', 'dance-electronic-digital-song-sales', 'dance-electronic-songs', 'dance-electronic-streaming-songs']
    electronic += ["DAN", "DAS", "DDT", "BSI", "BST", "DSI", "ELP"]
    if useElectronic:
        charts += electronic

    hot  = ['hot-100', 'pop-songs', 'radio-songs']
    hot += ['streaming-songs', 'rhythmic-40', 'heatseekers-albums']    
    top  = ['billboard-200', 'artist-100', 'top-album-sales']
    hot += top
    hot += ["TLN", "HSB", "HSI", "TLP", "TSL", "TFM", "ATS"]
    if useHot:
        charts += hot

    rnb  = ['r-and-b-albums', 'r-and-b-hip-hop-digital-song-sales', 'r-and-b-hip-hop-streaming-songs', 'r-and-b-songs', 'r-b-hip-hop-albums', 'r-b-hip-hop-songs']
    rnb += ['rap-albums','rap-song', 'hot-r-and-b-hip-hop-airplay']
    rnb += ["RLP", "RAP", "RBL", "RBM", "RBS", "RBT", "BLP"]
    if useRnB:
        charts += rnb

    rock  = ['rock-airplay','hot-mainstream-rock-tracks', 'rock-albums','rock-digital-song-sales','rock-songs','rock-streaming-songs', 'hard-rock-albums']
    rock += ["RCK", "RKA", "RKT", "ROS", "RTT", "MTL", "ARK"]

    if useRock:
        charts += rock

    canadian = ['canadian-albums','canadian-hot-100']
    canadian += ["CAN", "CNA"]

    latin    = ['latin-airplay','latin-albums','latin-digital-song-sales','latin-pop-albums','latin-pop-songs','latin-songs','latin-streaming-songs']
    latin   += ["HTA", "HTL", "LSS", "LDT", "LPO", "LPP", "LCM"]
    mexican  = ['regional-mexican-albums','regional-mexican-songs']
    mexican += ["LRM", "LMX"]
    world    = ['china-v-chart', 'japan-hot-100','tropical-albums', 'tropical-songs','world-albums', 'reggae-albums']
    world   += ["LSA", "LTS", "JPN", "WLP", "JAH", "ARG"]
    if useNonUS:
        charts += canadian + latin + mexican + world

    holiday  = ['holiday-albums','holiday-season-digital-song-sales','holiday-songs','holiday-streaming-songs']
    holiday += ['XSS', "XML", "XDT", "ASX"]
    if useHoliday:
        charts += holiday

    classical  = ['classical-albums']
    classical += ['COA']
    if useClassical:
        charts += classical

    jazz  = ['jazz-albums', 'jazz-songs']
    jazz += ['JLS', 'JSI']
    if useJazz:
        charts += jazz

    comedy  = ['comedy-albums']
    comedy += ['GIG']
    if useComedy:
        charts += comedy

    newage  = ['new-age-albums']
    newage += ['NLP']
    if useNewAge:
        charts += newage

    general = ['digital-albums','digital-song-sales']
    general = ['HDS', 'DLP']
    if useGeneral:
        charts += general

    kids  = ['kids-albums']
    kids += ['KID']
    if useKids:
        charts += kids

    twitter  = ['twitter-emerging-artists', 'twitter-top-tracks', 'social-50']
    twitter += ['STM', 'SOC']
    if useTwitter:
        charts += twitter

    vinyl  = ['vinyl-albums', "catalog-albums", 'tastemaker-albums']
    vinyl += ["TAS", "VNL", "TLC"]
    if useVinyl:
        charts += vinyl

    rare  = ['independent-albums']
    rare += ['IND']
    if useRare:
        charts += rare

    soundtrack = ['soundtracks']
    soundtrack += ["STX"]
    if useSountrack:
        charts += soundtrack
        
    other = []
    if useOther:
        charts += other  
        
    if useTest:
        charts += []
        
    if albums:
        charts = [x for x in charts if x.endswith("albums")]
        
    
    print("Charts: {0}".format(charts))
    
    artistRenames = getArtistRenames()
    renameStats   = Counter()
    fullChartData = {}
    chartData     = {}
    #country       = "USA"
    country       = country
    chartCounter  = Counter()

    for ifile in files:
        print(ifile,end='\t')
        fdata = getFile(ifile)
        for chartName, cnameResults in fdata.items():
            for date, dResults in cnameResults.items():
                if minYear is not None:
                    if getDateTime(date).year < int(minYear):
                        continue
                if maxYear is not None:
                    if getDateTime(date).year > int(maxYear):
                        continue
                      
                        

                if chartName not in charts:
                    continue
                    
                if False:
                    if chartName in nonMusic:
                        continue        
                    if chartName in mixMusic:
                        continue
                    if chartName.startswith("gospel"):
                        continue
                    if chartName.startswith("bluegrass"):
                        continue
                    if chartName.startswith("classical"):
                        continue
                    if chartName.startswith("jazz"):
                        continue
                    if chartName.startswith("holiday"):
                        continue
                    if chartName.startswith("comedy"):
                        continue
                    if chartName.startswith("soundtracks"):
                        continue
                    if chartName.startswith("new-age"):
                        continue
                    if chartName.startswith("heatseekers"):
                        continue
                    if chartName.startswith("dance"):
                        continue
                    if chartName.startswith("christian"):
                        continue
                    if chartName.startswith("kids"):
                        continue
                    if country in ["USA", "US"]:
                        if chartName in nonUS:
                            continue

                artist = dResults["Artist"]

                renamedArtist = artist
                for testArtist in artistRenames.keys():
                    if artist.find(testArtist) != -1:
                        tmp = renamedArtist
                        renamedArtist = renamedArtist.replace(testArtist, artistRenames.get(testArtist))
                        #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist   

                ignoreStatus = getArtistIgnores(artist)
                if ignoreStatus is False:
                    continue

                    
                chartCounter[chartName] += 1

                album  = dResults["Name"]
                
                if chartData.get(artist) is None:
                    chartData[artist] = Counter()
                chartData[artist][album] += 1


                
                
                if fullChartData.get(artist) is None:
                    fullChartData[artist] = {"Songs": {}, "Albums": {}}
                if chartName.endswith("Albums"):
                    key = "Albums"
                else:
                    key = "Songs"
                if fullChartData[artist][key].get(album) is None:
                    fullChartData[artist][key][album] = {}
                if fullChartData[artist][key][album].get(chartName) is None:
                    fullChartData[artist][key][album][chartName] = {}
                fullChartData[artist][key][album][chartName][date] = 0
        print(len(fullChartData))



    artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in fullChartData.items()}        

    if artistRenames is not None:
        print("Renamed {0} artists".format(len(renameStats)))
        print("Most Common Artists:")
        for item in renameStats.most_common(5):
            print(item)

    saveFile(idata=chartCounter, ifile="chartCounter.p", debug=True)
    saveFile(idata=chartData, ifile="chartData.p", debug=True)


    saveFile(idata=artistAlbumData, ifile="currentBillboardArtistAlbumData.p", debug=True)
    saveFile(idata=fullChartData, ifile="currentBillboardFullChartArtistAlbumData.p", debug=True)

    _, _ = clock("Last Run")

In [191]:
findArtistData(minYear=198, maxYear=2019, country="US", albums=False)

Charts: ['hot-100', 'pop-songs', 'radio-songs', 'streaming-songs', 'rhythmic-40', 'heatseekers-albums', 'billboard-200', 'artist-100', 'top-album-sales', 'TLN', 'HSB', 'HSI', 'TLP', 'TSL', 'TFM', 'ATS']
/Volumes/Piggy/Charts/data/billboard/results/1958.p	8
/Volumes/Piggy/Charts/data/billboard/results/1959.p	

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/utils-0.0.1-py3.7.egg/ioUtils.py:72: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


21
/Volumes/Piggy/Charts/data/billboard/results/1960.p	35
/Volumes/Piggy/Charts/data/billboard/results/1961.p	52
/Volumes/Piggy/Charts/data/billboard/results/1962.p	66
/Volumes/Piggy/Charts/data/billboard/results/1963.p	87
/Volumes/Piggy/Charts/data/billboard/results/1964.p	101
/Volumes/Piggy/Charts/data/billboard/results/1965.p	116
/Volumes/Piggy/Charts/data/billboard/results/1966.p	134
/Volumes/Piggy/Charts/data/billboard/results/1967.p	144
/Volumes/Piggy/Charts/data/billboard/results/1968.p	159
/Volumes/Piggy/Charts/data/billboard/results/1969.p	172
/Volumes/Piggy/Charts/data/billboard/results/1970.p	189
/Volumes/Piggy/Charts/data/billboard/results/1971.p	204
/Volumes/Piggy/Charts/data/billboard/results/1972.p	228
/Volumes/Piggy/Charts/data/billboard/results/1973.p	248
/Volumes/Piggy/Charts/data/billboard/results/1974.p	273
/Volumes/Piggy/Charts/data/billboard/results/1975.p	296
/Volumes/Piggy/Charts/data/billboard/results/1976.p	314
/Volumes/Piggy/Charts/data/billboard/results/1977

# Chart Analysis

In [193]:
chartCounter = getFile("chartCounter.p")
keys = chartCounter.keys()
sorted(keys)
#chartCounter.most_common()

['ATS',
 'HSB',
 'HSI',
 'TFM',
 'TLN',
 'TLP',
 'TSL',
 'artist-100',
 'billboard-200',
 'heatseekers-albums',
 'hot-100',
 'pop-songs',
 'radio-songs',
 'rhythmic-40',
 'streaming-songs',
 'top-album-sales']

In [78]:

slimResults = Counter()
for artist, artistData in results.items():
    for key, keyData in artistData.items():
        for album, albumData in keyData.items():
            slimResults[artist] += sum({k: len(v) for k,v in albumData.items()}.values())

In [ ]:
saveFile(idata = slimResults, ifile="billboardCounter.p", debug=True)
saveFile(idata = results,     ifile="billboardResults.p", debug=True)

# Get Billboard Results

In [ ]:
slimResults = getFile(ifile="billboardCounter.p", debug=True)
fullResults = getFile(ifile="billboardResults.p", debug=True)

# Get Discogs

In [ ]:
disc = discogs()
discdf = disc.getMasterSlimArtistDiscogsDB()
artistIDToName = discdf["DiscArtist"].to_dict()
from masterdb import getArtistAlbumsDB, discConv
artistAlbumsDB = getArtistAlbumsDB(disc)

artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

# Check Renames

In [ ]:
singleRenames = getFile(ifile="singleRenames.p", debug=True)
multiRenames  = getFile(ifile="multiRenames.p", debug=True)
knownArtists  = getFile(ifile="artistMap.p", debug=True)

# Near Renames Artists

In [ ]:
from searchUtils import findNearest
cutoff = 0.9
artistsToGet = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
#for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)):
    if i <= 4500:
        continue
    if i % 250 == 0:
        print("==>",i,len(slimResults))
    if multiRenames.get(artist) is not None:
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if knownArtists.get(artist) is not None:
        continue
    if manualRenames.get(artist) is not None:
        continue

    matches = mulArts.getArtistNames(artist)
    if len(matches) > 1:
        continue

    mdata = getMusicData("DiscArtist", artist)
    if mdata is None:
        results = findNearest(artist, artistNameToID.keys(), num=1, cutoff=cutoff)      
        if len(results) > 0:
            artistsToGet[artist] = results
            print("manualRenames[\"{0}\"] = \"{1}\"".format(artist, results[0]))

In [ ]:
findNearest

In [ ]:
for k,v in artistsToGet.items():
    print("keep[\"{0}\"] = {1}  #{2}".format(k, v[0], v[1]))



# Get Multi Results

In [ ]:
from collections import Counter
cutoff = 0.8
multiMatchResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    if manualRenames.get(prevArtist) is not None:
        prevArtist = manualRenames[prevArtist]
    mdata = getMusicData("DiscArtist", prevArtist)
    if isinstance(mdata, DataFrame):
        if mdata.shape[0] <= 1:
            continue
        matches = mdata["Name"].index
        artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
        results = getBestArtistIDMatch(artist, artistAlbums, matches, N=3, cutoff=cutoff)
        if results[2] is not None:
            if results[2] >= cutoff:
                print(i,"/",len(slimResults),'  \t',artist,results)
                multiMatchResult[artist] = list(results[:-1])

In [ ]:
len(multiMatchResult)

In [ ]:
print(len(knownArtists))
knownArtists.update(multiMatchResult)
print(len(knownArtists))
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

# Single Results

In [ ]:
knownArtists["The Kingston Trio"]

In [ ]:
cutoff = 0.60
singleResult = {}
#for i, (artist, cnt) in enumerate(slimResults.most_common()):
for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)):
    if i > 0 and i % 100 == 0 or i == 100:
        print("Passed",i,'/',len(slimResults))
    if cnt < 1:
        continue
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]        
    if prevArtist is None:
        prevArtist = artist
    if manualRenames.get(prevArtist) is not None:
        prevArtist = manualRenames[prevArtist]
    if knownArtists.get(prevArtist) is not None or knownArtists.get(artist) is not None:
        continue
    artistAlbums = set(list(fullResults[artist]["Songs"].keys()) + list(fullResults[artist]["Albums"].keys()))
    match = mulArts.getArtistNames(artist                                                     )
    if len(match) == 1:
        mdata = getMusicData("DiscArtist", list(match.keys())[0])
        if not isinstance(mdata, DataFrame):
            results = getBestArtistMatch(artist, artistAlbums, N=3, cutoff=cutoff)
            if results[2] is not None:
                if results[2] >= cutoff:
                    print(i,"/",len(slimResults),'  \t',artist,'\t',results)
                    singleResult[artist] = results
                    if len(singleResult) > 1000:
                        break

In [ ]:
del singleResult["Murk vs. Kristine W"]
del singleResult["Chris Cox Vs. Happy Clappers"]
del singleResult["T.M.Revolution × Nana Mizuki"]
del singleResult["Bob Marley Vs. Funkstar De Luxe"]

In [ ]:
singleResults = {k: v[0] for k,v in singleResult.items()}
print("Found {0} single results".format(len(singleResults)))
print("Found {0} all results".format(len(knownArtists)))
knownArtists.update(singleResults)
print("Found {0} all results".format(len(knownArtists)))
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

## Known Results

In [ ]:
from collections import Counter
cutoff = 0.8
multiMatchResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    mdata = getMusicData("DiscArtist", prevArtist)
    if isinstance(mdata, DataFrame):
        if mdata.shape[0] <= 1:
            continue
        matches = mdata["Name"].index
        artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
        results = getBestArtistIDMatch(artist, artistAlbums, matches, N=3, cutoff=cutoff)
        if results[2] is not None:
            if results[2] >= cutoff:
                print(i,"/",len(slimResults),'  \t',artist,results)
                multiMatchResult[artist] = list(results[:-1])

In [ ]:
singleResults = {k: v[0] for k,v in multiMatchResult.items()}
print("Found {0} single results".format(len(singleResults)))
print("Found {0} all results".format(len(knownArtists)))
knownArtists.update(singleResults)
print("Found {0} all results".format(len(knownArtists)))

In [ ]:
saveFile(idata=knownArtists, ifile="singleRenames.p")

# Check For Misnames

In [ ]:
ma = multiartist(cutoff=0.9, discdata=discdata)

In [ ]:
print(ma.getArtistNames('Lipps, Inc.') == {'Lipps, Inc.': ['159617', '26641', '101850']})
print(ma.getArtistNames('Bob Wills & His Texas Playboys') == {'Bob Wills & His Texas Playboys': ['670000','786114','1134146','804668','804679','1004309','935875','935907']})
print(ma.getArtistNames('Brad Paisley Duet With Carrie Underwood', debug=False) == {'Carrie Underwood': ['1011680'], 'Brad Paisley': ['313755']})
print(ma.getArtistNames('Ray Charles With Willie Nelson', debug=False) == {'Willie Nelson': ['249449'], 'Ray Charles': ['521963', '30552']})

# Album Data

In [ ]:
savename = join(savedir, "billboard.p")
albumData = getFile(join(savedir, "billboard-album.p"))
#chartdata = getFile(savename, debug=True)

In [ ]:
for artist,artistData in albumData.items():
    print(artist)
    for album in artistData.keys():
        print("\t",album)

# Show Data

In [ ]:
togets = {}
combos = []

savedir  = join(basedir, "results")
savename = join(savedir, "billboard.p")
chartdata = getFile(savename, debug=True)
chartArtists = sorted(chartdata.keys(), reverse=True)

import json
prevs = json.load(open("prevs.json", "r"))
artPrev = json.load(open("../discogs/prevs.json", "r"))
prevs.update(artPrev)
json.dump(prevs, open("prevs.json", "w"))
print("There are {0} artists previously searched.".format(len(prevs)))

ma = multiartist(cutoff=0.9, discdata=discdata, exact=False)

from random import shuffle
print("There are {0} chart artists".format(len(chartArtists)))
shuffle(chartArtists)
for i,artist in enumerate(chartArtists):
    if prevs.get(artist) is not None:
        continue
    retval = ma.getArtistNames(artist)
    if retval is None:
        togets[artist] = artist
    else:
        for name,value in retval.items():
            if prevs.get(name) is not None:
                continue
            if value is None:
                togets[name.upper()] = artist
                print("{0: <3}{1: <40}{2: <60}{3}".format(len(togets),name,artist,retval))
                print('\t---->',i,'/',len(chartArtists))

    
    if len(togets) > 50:
        print("\n\n\n\n")
        for name,artist in togets.items():
            print("art.searchDiscogForArtist(\"{0}\")  ## {1}".format(name, artist))
        break

###### artists = [x.strip() for x in vals.split("\n")]
artists = [x for x in artists if len(x) > 0]

In [ ]:
Pitbull Featuring Ne-Yo, Afrojack & Nayer
Drake Featuring Kanye West, Lil Wayne & Eminem

In [ ]:
def testName(name, discdata):
    if discdata.get(name) is not None:
        return discdata[name]
    return name

def findName(name, discArtists, threshold):
    retval = findNearest(name, discArtists, 1, threshold)
    if len(retval) == 0:
        return name
    return retval
    

def splitArtist(name):
    names = [name]
    names = [[x.strip() for x in y.split(" Featuring ")]
results = {}
    
for name in artists:
    if " & " in name:
        retvals = [testName(x, discdata) for x in 
        retvals = [findName(x, discArtists, 0.95) if not isinstance(x, list) else x for x in retvals]
        results[name] = retvals        

In [ ]:
togets = []
for k,v in results.items():
    for val in v:
        if not isinstance(val, list):
            togets.append("art.searchDiscogForArtist(\"{0}\")".format(val))
            
for toget in set(togets):
    print(toget)

In [ ]:
tmp = [['5070865', '90037', '4135543', '1507065'], ['Katy XXX Perry', ['1201210']]]

In [ ]:
tmp

In [ ]:
nameids = {}
namerefs = {}
for name,nameid in iddata.items():
    artist = name
    if name.endswith(")"):
        artist = None
        for x in [-3,-4,-5]:
            if artist is not None:
                continue
            if abs(x) > len(name):
                continue
            if name[x] == "(":
                try:
                    val = int(name[(x+1):-1])
                    artist = name[:x].strip()
                except:
                    continue
          
        if artist is None:
            artist = name

    ref = refdata[name]
            
    if nameids.get(artist) is None:
        nameids[artist] = {}
    nameids[artist][nameid] = 1
    
    if namerefs.get(artist) is None:
        namerefs[artist] = {}
    namerefs[artist][ref] = 1
    

nameids = {k: list(v.keys()) for k,v in nameids.items()}
namerefs = {k: list(v.keys()) for k,v in namerefs.items()}